# DATA522 Final Exploration

In [2]:
# Install libraries for reading in Parquet and OCR
%pip install pyarrow
%pip install ocrmac
%pip install pytesseract
%pip install easyocr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 22.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 24.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 15.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 24.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [8]:
pd.read_parquet('data/train-00000-of-00006-2c5eee001709386a.parquet')

,image,panel_name,characters,themes,color,year,caption
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19501002_00,"[Charlie Brown, Patty, Shermy]",[],grayscale,1950,charlie brown and peanuts cartoon
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19501002_01,"[Charlie Brown, Patty, Shermy]",[],grayscale,1950,snoopy's peanuts cartoon how i hate him
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19501002_02,"[Charlie Brown, Patty, Shermy]",[],grayscale,1950,good ol' charlie brown yes sir
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19501002_03,"[Charlie Brown, Patty, Shermy]",[],grayscale,1950,good ol' charlie brown
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19501003_00,"[Charlie Brown, Patty]",[],grayscale,1950,little girls are made of sugar and spice
...,...,...,...,...,...,...,...
12905,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19580615_04,"[Linus van Pelt, Snoopy]",[],grayscale,1958,here comes the fierce tiger screaming up on th...
12906,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19580615_05,"[Linus van Pelt, Snoopy]",[],grayscale,1958,snoopy sleeping in the grass
12907,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19580615_06,"[Linus van Pelt, Snoopy]",[],grayscale,1958,snoopy and his dog are playing in the grass
12908,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,19580616_00,[Charlie Brown],[],grayscale,1958,peanuts so drop the fly ball and we lose the c...
